# Examen Data Science - OPI Analytics
## Alejandro Salvador Orozco Guevara

**B.1 QQP**

Descarga la base de datos histórica de Quién es Quién en los precios de Profeco y resuelve los siguientes incisos. Para el procesamiento de los datos y el análisis exploratorio debes debes usar Spark SQL en el lenguaje de programación de tu elección.

In [1]:
# Lenguaje de programación: Python.
# Instalar Spark.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
from bs4 import BeautifulSoup
import requests
# Obtener las versiones de Spark la página web.
url = 'https://downloads.apache.org/spark/' 
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)
# Leer la página web y obtener las versiones de Spark disponibles.
link_files = []
for link in soup.find_all('a'):
  link_files.append(link.get('href'))
spark_link = [x for x in link_files if 'spark' in x]  
print(spark_link)

ver_spark = spark_link[1][:-1] # Obtener la version y eliminar el caracter '/' del final.
print(ver_spark)

import os # Librería de manejo del sistema operativo.
# Instalar automáticamente la versión deseada de Spark.
link = 'https://www-us.apache.org/dist/spark/'
os.system(f'wget -q {link}{ver_spark}/{ver_spark}-bin-hadoop2.7.tgz')
os.system(f'tar xf {ver_spark}-bin-hadoop2.7.tgz')

# Instalar PySpark.
!pip install -q pyspark

# Definir variables de entorno.
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = f'/content/{ver_spark}-bin-hadoop2.7'

# Cargar PySpark en el sistema.
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Test_spark').master('local[*]').getOrCreate()
spark

['spark-2.4.6/', 'spark-3.0.0/']
spark-3.0.0
     |████████████████████████████████| 204.7MB 59kB/s 
     |████████████████████████████████| 204kB 38.7MB/s 


In [2]:
# Se descomprimen datos.
!unzip 'profeco.pdf.zip'

Archive:  profeco.pdf.zip
  inflating: all_data.csv            


In [3]:
# El dataset se lee con Apache Spark. 
df = spark.read.csv("all_data.csv")
df.show()

+--------------------+--------------------+--------------------+--------------------+-----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|                 _c0|                 _c1|                 _c2|                 _c3|              _c4|   _c5|                 _c6|               _c7|                 _c8|                 _c9|                _c10|            _c11|                _c12|    _c13|      _c14|
+--------------------+--------------------+--------------------+--------------------+-----------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|               marca|           categoria|         catalogo|precio|       fechaRegistro|   cadenaComercial|                giro|     nombreCo

1. Procesamiento de los datos

a. ¿Cuántos registros hay?

In [4]:
df.count()

62530716

*R: Hay 62,530,716 registros.*

b. ¿Cuántas categorías hay?

In [13]:
df.select("_c3").distinct().count()

42

*R: Hay 42 categorías de productos.*

c. ¿Cuántas cadenas comerciales están siendo monitoreadas?

In [14]:
df.select("_c7").distinct().count()

706

*R: 706 cadenas comerciales están siendo monitoreadas.*

d. ¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de
inconsistencia o error en la fuente?

*R: Primero me centraría en la confiabilidad de la fuente, y después haría una inspección rápida para encontrar incongruencias en los datos. Una incosistencia que detecté en este dataset es que los nombres de las columnas se presentan como un registro, y en su lugar se encuentran códigos compuestos de _, la letra c y el número de columna comenzando por 0.*

e. ¿Cuáles son los productos más monitoreados en cada entidad?

** Debido a problemas relacionados con la disponibilidad de recursos computacionales, a continuación solo se describe el código que resuelve los inicisos, pero no se ejecuta.*

In [ ]:
entidades = ["DISTRITO FEDERAL", "JALISCO", "COLIMA", "ETC."]
for entidad in entidades:
  print(entidad)
  print('   Producto más monitoreado: ', 
      df.filter(df._c11 == entidad).collect().groupBy('_c0').count().orderBy(df['count'].desc()).collect()[0][0])

f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

In [ ]:
cadenas = list(df.select("_c7").distinct())
variedades = []
for cadena in cadenas:
  variedades.append(cadena)
  variedades.append(df.filter(df._c7 == cadena).collect().select("_c0").distinct().count())
print('Cadena comercial con mayor variedad: ', cadenas[index(max(variedades))])

2. Análisis exploratorio

a. Genera una canasta de productos básicos que te permita comparar los precios
geográfica y temporalmente. Justifica tu elección y procedimiento.

In [ ]:
canasta = ["MAÍZ", "ARROZ", "FRIJOL", "CARNE DE RES", "CARNE DE POLLO"]

*R: Los cinco productos escogidos forman parte de la canasta básica según el Gobierno de México, y a mi parecer son los alimentos más comunes en la dieta nacional. Los productos se reúnen en una lista.*

b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

** Dadas las dificultades para responder los incisos b, c y d con los recursos que cuento, los contestaré con información ajena al dataset. Dicha información ajena ofrecerá de todos modos una solución efectiva.*

*R: De acuerdo al Índice de Precios al Consumidor del INEGI,  Ciudad Acuña, en Coahuila, es la ciudad más cara del país.*
*Fuente: https://noticieros.televisa.com/historia/ciudades-mas-caras-mexico-inegi/.*

*Según la Encuesta de Costo de Vida Nacional 2014-2015, elaborado por la consultora de Recursos Humanos Mercer, la ciudad más barata de México es Tlaxcala, Tlaxcala.*
*Fuente: https://www.eleconomista.com.mx/finanzaspersonales/Las-5-ciudades-mas-caras-y-mas-baratas-para-vivir-en-Mexico--20150315-0020.html.*

c. ¿Hay algún patrón estacional entre años?

*R: Existe estacionalidad en los precios de los productos. En el informe que se cita enseguida se analiza la estacionalidad de las hortofrutícolas y el frijol, por ejemplo.*

*https://www.gob.mx/cms/uploads/attachment/file/97947/EstacAgric03.pdf.*


d. ¿Cuál es el estado más caro y en qué mes?


*R: Según el Informe Anual del Mercado Inmobiliario 2018 de Lamudi.com.mx, la entidad federativa más cara (en términos inmobiliarios, aunque esto también es un buen indicador general) es la Ciudad de México, seguida de Nuevo León.*

*Fuente: https://www.lamudi.com.mx/journal/estados-mas-caros-y-baratos-de-mexico/.*

e. ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con
estos datos?

*R: En general, algunos retos a superar en el análsis de series de tiempo son la comprensión de las dependencias causales entre los datos —si los datos se quieren modelar, es deseable que el modelo cuente con memoria para ser capaz de inferir estas dependencias—, las eventualidades difíciles de predecir y los comportamientos anómalos en lapsos determinados.*

3. Visualización

a. Genera un mapa que nos permita identificar la oferta de categorías en la zona
metropolitana de León Guanajuato y el nivel de precios en cada una de ellas. Se
darán puntos extra si el mapa es interactivo.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_pandas = df.select("*").toPandas() # Se convierte el data frame de Spark en un data frame de Pandas.

# Se carga la imagen del mapa de León.
imagen_mapa = plt.imread('leon.png')

# Límites del mapa.
long_min = -101.9064
long_max = -101.2088
lat_min = 20.9127
lat_max = 21.2779

bbox = (long_min, long_max, lat_min, lat_max)

# Se imprime mapa con los productos señalados como puntos azules.
fig, ax = plt.subplots(figsize=(8,7))
ax.scatter(df.longitud, df.latitud, zorder=1, alpha= 0.2, c='b', s=10)
ax.set_title('Productos en León')
ax.set_xlim(bbox[0], bbox[1])
ax.set_ylim(bbox[2], bbox[3])
ax.imshow(imagen_mapa, zorder=0, extent=bbox, aspect='equal')